In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [3]:
fips_check = pd.read_csv("data/state_data/geo/geo_ar.csv") #- original geocoded file
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head() #5857

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,13588,13588,5336097104,4/13/20,669,PPP,"MCC GROUP HOLDINGS, INC.",2300 North Lowell Road,Springdale,AR,...,Unanswered,NaN,10113698.63,6/11/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089
1,13589,13589,5690507110,4/13/20,669,PPP,SYSTEMS PLANT SERVICES INC,POBOX 11390,EL DORADO,AR,...,Unanswered,NaN,10097165.73,6/25/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267
2,13590,13590,2286469010,5/15/21,669,PPP,CARTI,8901 Carti Way,Little Rock,AR,...,Unanswered,Y,NaN,NaN,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816
3,13591,13591,8080007000,4/8/20,669,PPP,"MEDICAL ASSOCIATES OF NWA, P.A.",3383 N MANA CT SUITE 201,FAYETTEVILLE,AR,...,Unanswered,NaN,7347112.11,7/1/21,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307
4,13592,13592,5612127108,4/13/20,669,PPP,"THE PRICE COMPANIES, INC.","218 Midway Route,",Monticello,AR,...,Unanswered,NaN,7314049.23,6/11/21,32.0,Manufacturing,ARKANSAS DISTRICT OFFICE,"218 Midway Route,, Monticello, AR",33.605533,-91.786633


In [4]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [5]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #2

,LoanNumber,full_add,Lat,Long
3039,2991708608,"720 Ramsey St, Batesville, AR",NaN,NaN
5177,4507858300,"528 Central Ave Ste 100, Hot Springs National ...",NaN,NaN


In [11]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #5855

,LoanNumber,full_add,Lat,Long
0,5336097104,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089
1,5690507110,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267
2,2286469010,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816
3,8080007000,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307
4,5612127108,"218 Midway Route,, Monticello, AR",33.605533,-91.786633


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [34]:
fips_check_scraped = pd.read_csv("data/state_data/geo/geo_fips/AR_fips_scraped.csv") 
fips_check_scraped.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,13588,13588,5336097104,4/13/20,669,PPP,"MCC GROUP HOLDINGS, INC.",2300 North Lowell Road,Springdale,...,NaN,10113698.63,6/11/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089,5.143010e+13
1,1,13589,13589,5690507110,4/13/20,669,PPP,SYSTEMS PLANT SERVICES INC,POBOX 11390,EL DORADO,...,NaN,10097165.73,6/25/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267,5.139951e+13
2,2,13590,13590,2286469010,5/15/21,669,PPP,CARTI,8901 Carti Way,Little Rock,...,Y,NaN,NaN,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816,5.119002e+13
3,3,13591,13591,8080007000,4/8/20,669,PPP,"MEDICAL ASSOCIATES OF NWA, P.A.",3383 N MANA CT SUITE 201,FAYETTEVILLE,...,NaN,7347112.11,7/1/21,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307,5.143011e+13
4,4,13592,13592,5612127108,4/13/20,669,PPP,"THE PRICE COMPANIES, INC.","218 Midway Route,",Monticello,...,NaN,7314049.23,6/11/21,32.0,Manufacturing,ARKANSAS DISTRICT OFFICE,"218 Midway Route,, Monticello, AR",33.605533,-91.786633,5.043490e+13


In [35]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [36]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [37]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() #15792

,LoanNumber,full_add,Lat,Long,FIPS_z
0,5336097104,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089,5.143010e+13
1,5690507110,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267,5.139951e+13
2,2286469010,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816,5.119002e+13
3,8080007000,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307,5.143011e+13
4,5612127108,"218 Midway Route,, Monticello, AR",33.605533,-91.786633,5.043490e+13


In [39]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #659

,LoanNumber,full_add,Lat,Long,FIPS_z
605,5028007010,"1402 S 1ST ST, MC GEHEE, AR",30.250536,-97.755145,NaN
3091,8762207009,"E & M Contractors, LLC, PO Box 9287, AR",40.021671,-75.946420,NaN
4316,7127367007,"19190 AR HIGHWAY 59, SILOAM SPRINGS, AR",40.110548,-95.145778,NaN
4672,8811267002,"2179 CR 321, Poplar Bluff, AR",36.682890,-90.465882,NaN
5202,7415727100,"800 West Ninth, Little Rock, AR",34.741746,-92.280386,NaN
...,...,...,...,...,...
5852,7337827110,"117 S Richards Street,, Benton, AR",34.562906,-92.593465,NaN
5853,7492638402,"20670 Interstate 30 N, Benton, AR",34.575929,-92.586555,NaN
5854,7733137106,"11125 Arcade Dr suite G, LITTLE ROCK, AR",34.769251,-92.397557,NaN
5855,9580078508,"1753 BUTTFIELD COACH RD, SPRINGDALE, AR",36.168671,-94.094005,NaN


<h4>FIPS Script

In [40]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [41]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('05')] #AR
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [42]:
geo_id_list =geo_id 

Length Check

In [43]:
print(len(geo_id_list))

659


Matching FIPS to DataFrame

In [44]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:5200] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null

In [45]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

<ipython-input-45-08e9c917fe2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
605,5028007010,"1402 S 1ST ST, MC GEHEE, AR",30.250536,-97.755145,[]
3091,8762207009,"E & M Contractors, LLC, PO Box 9287, AR",40.021671,-75.946420,[]
4316,7127367007,"19190 AR HIGHWAY 59, SILOAM SPRINGS, AR",40.110548,-95.145778,[]
4672,8811267002,"2179 CR 321, Poplar Bluff, AR",36.682890,-90.465882,[]
5202,7415727100,"800 West Ninth, Little Rock, AR",34.741746,-92.280386,[051190045002050]
...,...,...,...,...,...
5852,7337827110,"117 S Richards Street,, Benton, AR",34.562906,-92.593465,[051250106013002]
5853,7492638402,"20670 Interstate 30 N, Benton, AR",34.575929,-92.586555,[051250101011005]
5854,7733137106,"11125 Arcade Dr suite G, LITTLE ROCK, AR",34.769251,-92.397557,[051190022091010]
5855,9580078508,"1753 BUTTFIELD COACH RD, SPRINGDALE, AR",36.168671,-94.094005,[051430101012019]


Remove brackets

In [46]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

<ipython-input-46-e19831f223ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
605,5028007010,"1402 S 1ST ST, MC GEHEE, AR",30.250536,-97.755145,NaN
3091,8762207009,"E & M Contractors, LLC, PO Box 9287, AR",40.021671,-75.946420,NaN
4316,7127367007,"19190 AR HIGHWAY 59, SILOAM SPRINGS, AR",40.110548,-95.145778,NaN
4672,8811267002,"2179 CR 321, Poplar Bluff, AR",36.682890,-90.465882,NaN
5202,7415727100,"800 West Ninth, Little Rock, AR",34.741746,-92.280386,051190045002050
...,...,...,...,...,...
5852,7337827110,"117 S Richards Street,, Benton, AR",34.562906,-92.593465,051250106013002
5853,7492638402,"20670 Interstate 30 N, Benton, AR",34.575929,-92.586555,051250101011005
5854,7733137106,"11125 Arcade Dr suite G, LITTLE ROCK, AR",34.769251,-92.397557,051190022091010
5855,9580078508,"1753 BUTTFIELD COACH RD, SPRINGDALE, AR",36.168671,-94.094005,051430101012019


In [47]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #4

,LoanNumber,full_add,Lat,Long,FIPS_z
605,5028007010,"1402 S 1ST ST, MC GEHEE, AR",30.250536,-97.755145,NaN
3091,8762207009,"E & M Contractors, LLC, PO Box 9287, AR",40.021671,-75.946420,NaN
4316,7127367007,"19190 AR HIGHWAY 59, SILOAM SPRINGS, AR",40.110548,-95.145778,NaN
4672,8811267002,"2179 CR 321, Poplar Bluff, AR",36.682890,-90.465882,NaN


Parse for Merge

In [71]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
605,5028007010,NaN
3091,8762207009,NaN
4316,7127367007,NaN
4672,8811267002,NaN
5202,7415727100,051190045002050
...,...,...
5852,7337827110,051250106013002
5853,7492638402,051250101011005
5854,7733137106,051190022091010
5855,9580078508,051430101012019


Read in base data

In [72]:
#if from geocode file
#fips_check = pd.read_csv("data/state_data/geo/geo_ar.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
#fips_check

#if from FIPS file
fips_check = pd.read_csv("data/state_data/geo/geo_fips/AR_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head() #15796


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,13588,13588,5336097104,4/13/20,669,PPP,"MCC GROUP HOLDINGS, INC.",2300 North Lowell Road,Springdale,...,NaN,10113698.63,6/11/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089,5.143010e+13
1,1,13589,13589,5690507110,4/13/20,669,PPP,SYSTEMS PLANT SERVICES INC,POBOX 11390,EL DORADO,...,NaN,10097165.73,6/25/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267,5.139951e+13
2,2,13590,13590,2286469010,5/15/21,669,PPP,CARTI,8901 Carti Way,Little Rock,...,Y,NaN,NaN,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816,5.119002e+13
3,3,13591,13591,8080007000,4/8/20,669,PPP,"MEDICAL ASSOCIATES OF NWA, P.A.",3383 N MANA CT SUITE 201,FAYETTEVILLE,...,NaN,7347112.11,7/1/21,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307,5.143011e+13
4,4,13592,13592,5612127108,4/13/20,669,PPP,"THE PRICE COMPANIES, INC.","218 Midway Route,",Monticello,...,NaN,7314049.23,6/11/21,32.0,Manufacturing,ARKANSAS DISTRICT OFFICE,"218 Midway Route,, Monticello, AR",33.605533,-91.786633,5.043490e+13


Merge with MainFrame

In [73]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,13588,13588,5336097104,4/13/20,669,PPP,"MCC GROUP HOLDINGS, INC.",2300 North Lowell Road,Springdale,...,10113698.63,6/11/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089,5.143010e+13,NaN
1,1,13589,13589,5690507110,4/13/20,669,PPP,SYSTEMS PLANT SERVICES INC,POBOX 11390,EL DORADO,...,10097165.73,6/25/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267,5.139951e+13,NaN
2,2,13590,13590,2286469010,5/15/21,669,PPP,CARTI,8901 Carti Way,Little Rock,...,NaN,NaN,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816,5.119002e+13,NaN
3,3,13591,13591,8080007000,4/8/20,669,PPP,"MEDICAL ASSOCIATES OF NWA, P.A.",3383 N MANA CT SUITE 201,FAYETTEVILLE,...,7347112.11,7/1/21,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307,5.143011e+13,NaN
4,4,13592,13592,5612127108,4/13/20,669,PPP,"THE PRICE COMPANIES, INC.","218 Midway Route,",Monticello,...,7314049.23,6/11/21,32.0,Manufacturing,ARKANSAS DISTRICT OFFICE,"218 Midway Route,, Monticello, AR",33.605533,-91.786633,5.043490e+13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5852,5852,19440,19440,7337827110,4/14/20,669,PPP,"N & D TECHNICAL SERVICES, INC","117 S Richards Street,",Benton,...,151754.79,6/24/21,54.0,"Professional, Scientific, and Technical Services",ARKANSAS DISTRICT OFFICE,"117 S Richards Street,, Benton, AR",34.562906,-92.593465,NaN,051250106013002
5853,5853,19441,19441,7492638402,2/12/21,669,PPS,JW & BL ENTERPRISES/BENTON AR. LLC,20670 Interstate 30 N,Benton,...,150431.51,6/25/21,72.0,Accomodation and Food Services,ARKANSAS DISTRICT OFFICE,"20670 Interstate 30 N, Benton, AR",34.575929,-92.586555,NaN,051250101011005
5854,5854,19442,19442,7733137106,4/14/20,669,PPP,REJUVENATION CLINIC INC,11125 Arcade Dr suite G,LITTLE ROCK,...,151886.30,7/20/21,44.0,Retail Trade,ARKANSAS DISTRICT OFFICE,"11125 Arcade Dr suite G, LITTLE ROCK, AR",34.769251,-92.397557,NaN,051190022091010
5855,5855,19443,19443,9580078508,3/12/21,669,PPS,DK DESIGN LLC,1753 BUTTFIELD COACH RD,SPRINGDALE,...,150875.34,10/25/21,54.0,"Professional, Scientific, and Technical Services",ARKANSAS DISTRICT OFFICE,"1753 BUTTFIELD COACH RD, SPRINGDALE, AR",36.168671,-94.094005,NaN,051430101012019


Fill FIPS Columns

In [74]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,13588,13588,5336097104,4/13/20,669,PPP,"MCC GROUP HOLDINGS, INC.",2300 North Lowell Road,Springdale,...,6/11/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089,5.143010e+13,NaN,5.14301e+13
1,1,13589,13589,5690507110,4/13/20,669,PPP,SYSTEMS PLANT SERVICES INC,POBOX 11390,EL DORADO,...,6/25/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267,5.139951e+13,NaN,5.13995e+13
2,2,13590,13590,2286469010,5/15/21,669,PPP,CARTI,8901 Carti Way,Little Rock,...,NaN,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816,5.119002e+13,NaN,5.119e+13
3,3,13591,13591,8080007000,4/8/20,669,PPP,"MEDICAL ASSOCIATES OF NWA, P.A.",3383 N MANA CT SUITE 201,FAYETTEVILLE,...,7/1/21,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307,5.143011e+13,NaN,5.14301e+13
4,4,13592,13592,5612127108,4/13/20,669,PPP,"THE PRICE COMPANIES, INC.","218 Midway Route,",Monticello,...,6/11/21,32.0,Manufacturing,ARKANSAS DISTRICT OFFICE,"218 Midway Route,, Monticello, AR",33.605533,-91.786633,5.043490e+13,NaN,5.04349e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5852,5852,19440,19440,7337827110,4/14/20,669,PPP,"N & D TECHNICAL SERVICES, INC","117 S Richards Street,",Benton,...,6/24/21,54.0,"Professional, Scientific, and Technical Services",ARKANSAS DISTRICT OFFICE,"117 S Richards Street,, Benton, AR",34.562906,-92.593465,NaN,051250106013002,051250106013002
5853,5853,19441,19441,7492638402,2/12/21,669,PPS,JW & BL ENTERPRISES/BENTON AR. LLC,20670 Interstate 30 N,Benton,...,6/25/21,72.0,Accomodation and Food Services,ARKANSAS DISTRICT OFFICE,"20670 Interstate 30 N, Benton, AR",34.575929,-92.586555,NaN,051250101011005,051250101011005
5854,5854,19442,19442,7733137106,4/14/20,669,PPP,REJUVENATION CLINIC INC,11125 Arcade Dr suite G,LITTLE ROCK,...,7/20/21,44.0,Retail Trade,ARKANSAS DISTRICT OFFICE,"11125 Arcade Dr suite G, LITTLE ROCK, AR",34.769251,-92.397557,NaN,051190022091010,051190022091010
5855,5855,19443,19443,9580078508,3/12/21,669,PPS,DK DESIGN LLC,1753 BUTTFIELD COACH RD,SPRINGDALE,...,10/25/21,54.0,"Professional, Scientific, and Technical Services",ARKANSAS DISTRICT OFFICE,"1753 BUTTFIELD COACH RD, SPRINGDALE, AR",36.168671,-94.094005,NaN,051430101012019,051430101012019


Drop Extra FIPS Columns

In [75]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,5336097104,4/13/20,669,PPP,"MCC GROUP HOLDINGS, INC.",2300 North Lowell Road,Springdale,AR,72764-1842,...,NaN,10113698.63,6/11/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089,5.14301e+13
1,1,5690507110,4/13/20,669,PPP,SYSTEMS PLANT SERVICES INC,POBOX 11390,EL DORADO,AR,71730,...,NaN,10097165.73,6/25/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267,5.13995e+13
2,2,2286469010,5/15/21,669,PPP,CARTI,8901 Carti Way,Little Rock,AR,72205-6523,...,Y,NaN,NaN,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816,5.119e+13
3,3,8080007000,4/8/20,669,PPP,"MEDICAL ASSOCIATES OF NWA, P.A.",3383 N MANA CT SUITE 201,FAYETTEVILLE,AR,72703-4960,...,NaN,7347112.11,7/1/21,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307,5.14301e+13
4,4,5612127108,4/13/20,669,PPP,"THE PRICE COMPANIES, INC.","218 Midway Route,",Monticello,AR,71655-8606,...,NaN,7314049.23,6/11/21,32.0,Manufacturing,ARKANSAS DISTRICT OFFICE,"218 Midway Route,, Monticello, AR",33.605533,-91.786633,5.04349e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5852,5852,7337827110,4/14/20,669,PPP,"N & D TECHNICAL SERVICES, INC","117 S Richards Street,",Benton,AR,72015-5142,...,NaN,151754.79,6/24/21,54.0,"Professional, Scientific, and Technical Services",ARKANSAS DISTRICT OFFICE,"117 S Richards Street,, Benton, AR",34.562906,-92.593465,051250106013002
5853,5853,7492638402,2/12/21,669,PPS,JW & BL ENTERPRISES/BENTON AR. LLC,20670 Interstate 30 N,Benton,AR,72019-8463,...,NaN,150431.51,6/25/21,72.0,Accomodation and Food Services,ARKANSAS DISTRICT OFFICE,"20670 Interstate 30 N, Benton, AR",34.575929,-92.586555,051250101011005
5854,5854,7733137106,4/14/20,669,PPP,REJUVENATION CLINIC INC,11125 Arcade Dr suite G,LITTLE ROCK,AR,72212-4088,...,NaN,151886.30,7/20/21,44.0,Retail Trade,ARKANSAS DISTRICT OFFICE,"11125 Arcade Dr suite G, LITTLE ROCK, AR",34.769251,-92.397557,051190022091010
5855,5855,9580078508,3/12/21,669,PPS,DK DESIGN LLC,1753 BUTTFIELD COACH RD,SPRINGDALE,AR,72765,...,NaN,150875.34,10/25/21,54.0,"Professional, Scientific, and Technical Services",ARKANSAS DISTRICT OFFICE,"1753 BUTTFIELD COACH RD, SPRINGDALE, AR",36.168671,-94.094005,051430101012019


In [76]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #verify #6

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
605,605,5028007010,4/4/20,669,PPP,"C B KING MEMORIAL SCHOOL, INC",1402 S 1ST ST,MC GEHEE,AR,71654-2926,...,Y,933939.40,8/17/21,61.0,Educational Services,ARKANSAS DISTRICT OFFICE,"1402 S 1ST ST, MC GEHEE, AR",30.250536,-97.755145,NaN
3039,3039,2991708608,3/16/21,669,PPS,WRD INC,720 Ramsey St,Batesville,AR,72501-7137,...,NaN,274269.52,6/23/21,33.0,Manufacturing,ARKANSAS DISTRICT OFFICE,"720 Ramsey St, Batesville, AR",NaN,NaN,NaN
3091,3091,8762207009,4/8/20,669,PPP,"E & M CONTRACTORS, LLC","E & M Contractors, LLC",PO Box 9287,AR,72403,...,NaN,270255.75,11/27/20,23.0,Construction,ARKANSAS DISTRICT OFFICE,"E & M Contractors, LLC, PO Box 9287, AR",40.021671,-75.946420,NaN
4316,4316,7127367007,4/7/20,669,PPP,FARLEY'S INCORPORATED,19190 AR HIGHWAY 59,SILOAM SPRINGS,AR,72761-5228,...,NaN,200133.63,1/12/21,33.0,Manufacturing,ARKANSAS DISTRICT OFFICE,"19190 AR HIGHWAY 59, SILOAM SPRINGS, AR",40.110548,-95.145778,NaN
4672,4672,8811267002,4/8/20,669,PPP,"CANE CREEK QUARRY, LLC",2179 CR 321,Poplar Bluff,AR,72450,...,NaN,174626.56,12/9/20,21.0,Mining,ARKANSAS DISTRICT OFFICE,"2179 CR 321, Poplar Bluff, AR",36.682890,-90.465882,NaN
5177,5177,4507858300,1/23/21,669,PPS,CATADORES JRR INC,528 Central Ave Ste 100,Hot Springs National Park,AR,71901-3597,...,NaN,NaN,NaN,72.0,Accomodation and Food Services,ARKANSAS DISTRICT OFFICE,"528 Central Ave Ste 100, Hot Springs National ...",NaN,NaN,NaN


In [78]:
fips_AR = fips_merge_drop

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,5336097104,4/13/20,669,PPP,"MCC GROUP HOLDINGS, INC.",2300 North Lowell Road,Springdale,AR,72764-1842,...,NaN,10113698.63,6/11/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089,5.14301e+13
1,1,5690507110,4/13/20,669,PPP,SYSTEMS PLANT SERVICES INC,POBOX 11390,EL DORADO,AR,71730,...,NaN,10097165.73,6/25/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267,5.13995e+13
2,2,2286469010,5/15/21,669,PPP,CARTI,8901 Carti Way,Little Rock,AR,72205-6523,...,Y,NaN,NaN,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816,5.119e+13
3,3,8080007000,4/8/20,669,PPP,"MEDICAL ASSOCIATES OF NWA, P.A.",3383 N MANA CT SUITE 201,FAYETTEVILLE,AR,72703-4960,...,NaN,7347112.11,7/1/21,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307,5.14301e+13
4,4,5612127108,4/13/20,669,PPP,"THE PRICE COMPANIES, INC.","218 Midway Route,",Monticello,AR,71655-8606,...,NaN,7314049.23,6/11/21,32.0,Manufacturing,ARKANSAS DISTRICT OFFICE,"218 Midway Route,, Monticello, AR",33.605533,-91.786633,5.04349e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5852,5852,7337827110,4/14/20,669,PPP,"N & D TECHNICAL SERVICES, INC","117 S Richards Street,",Benton,AR,72015-5142,...,NaN,151754.79,6/24/21,54.0,"Professional, Scientific, and Technical Services",ARKANSAS DISTRICT OFFICE,"117 S Richards Street,, Benton, AR",34.562906,-92.593465,051250106013002
5853,5853,7492638402,2/12/21,669,PPS,JW & BL ENTERPRISES/BENTON AR. LLC,20670 Interstate 30 N,Benton,AR,72019-8463,...,NaN,150431.51,6/25/21,72.0,Accomodation and Food Services,ARKANSAS DISTRICT OFFICE,"20670 Interstate 30 N, Benton, AR",34.575929,-92.586555,051250101011005
5854,5854,7733137106,4/14/20,669,PPP,REJUVENATION CLINIC INC,11125 Arcade Dr suite G,LITTLE ROCK,AR,72212-4088,...,NaN,151886.30,7/20/21,44.0,Retail Trade,ARKANSAS DISTRICT OFFICE,"11125 Arcade Dr suite G, LITTLE ROCK, AR",34.769251,-92.397557,051190022091010
5855,5855,9580078508,3/12/21,669,PPS,DK DESIGN LLC,1753 BUTTFIELD COACH RD,SPRINGDALE,AR,72765,...,NaN,150875.34,10/25/21,54.0,"Professional, Scientific, and Technical Services",ARKANSAS DISTRICT OFFICE,"1753 BUTTFIELD COACH RD, SPRINGDALE, AR",36.168671,-94.094005,051430101012019


Convert to File

In [4]:
fips_AR.to_csv("data/state_data/geo/geo_fips/AR_fips_scraped.csv") 

Review & Compare

In [2]:
geo_AR_fips = pd.read_csv("data/state_data/geo/geo_fips/AR_fips_scraped.csv") 
geo_AR_fips 

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,0,5336097104,4/13/20,669,PPP,"MCC GROUP HOLDINGS, INC.",2300 North Lowell Road,Springdale,...,N,10113698.63,6/11/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"2300 North Lowell Road, Springdale, AR",36.202943,-94.129089,5.143010e+13
1,1,1,1,5690507110,4/13/20,669,PPP,SYSTEMS PLANT SERVICES INC,POBOX 11390,EL DORADO,...,N,10097165.73,6/25/21,23.0,Construction,ARKANSAS DISTRICT OFFICE,"POBOX 11390, EL DORADO, AR",33.207630,-92.666267,5.139951e+13
2,2,2,2,2286469010,5/15/21,669,PPP,CARTI,8901 Carti Way,Little Rock,...,Y,NaN,NaN,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"8901 Carti Way, Little Rock, AR",34.743626,-92.367816,5.119002e+13
3,3,3,3,8080007000,4/8/20,669,PPP,"MEDICAL ASSOCIATES OF NWA, P.A.",3383 N MANA CT SUITE 201,FAYETTEVILLE,...,N,7347112.11,7/1/21,62.0,Health Care and Social Assistance,ARKANSAS DISTRICT OFFICE,"3383 N MANA CT SUITE 201, FAYETTEVILLE, AR",36.113035,-94.153307,5.143011e+13
4,4,4,4,5612127108,4/13/20,669,PPP,"THE PRICE COMPANIES, INC.","218 Midway Route,",Monticello,...,N,7314049.23,6/11/21,32.0,Manufacturing,ARKANSAS DISTRICT OFFICE,"218 Midway Route,, Monticello, AR",33.605533,-91.786633,5.043490e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5852,5852,5852,5852,7337827110,4/14/20,669,PPP,"N & D TECHNICAL SERVICES, INC","117 S Richards Street,",Benton,...,N,151754.79,6/24/21,54.0,"Professional, Scientific, and Technical Services",ARKANSAS DISTRICT OFFICE,"117 S Richards Street,, Benton, AR",34.562906,-92.593465,5.125011e+13
5853,5853,5853,5853,7492638402,2/12/21,669,PPS,JW & BL ENTERPRISES/BENTON AR. LLC,20670 Interstate 30 N,Benton,...,N,150431.51,6/25/21,72.0,Accomodation and Food Services,ARKANSAS DISTRICT OFFICE,"20670 Interstate 30 N, Benton, AR",34.575929,-92.586555,5.125010e+13
5854,5854,5854,5854,7733137106,4/14/20,669,PPP,REJUVENATION CLINIC INC,11125 Arcade Dr suite G,LITTLE ROCK,...,N,151886.30,7/20/21,44.0,Retail Trade,ARKANSAS DISTRICT OFFICE,"11125 Arcade Dr suite G, LITTLE ROCK, AR",34.769251,-92.397557,5.119002e+13
5855,5855,5855,5855,9580078508,3/12/21,669,PPS,DK DESIGN LLC,1753 BUTTFIELD COACH RD,SPRINGDALE,...,N,150875.34,10/25/21,54.0,"Professional, Scientific, and Technical Services",ARKANSAS DISTRICT OFFICE,"1753 BUTTFIELD COACH RD, SPRINGDALE, AR",36.168671,-94.094005,5.143010e+13


In [3]:
#fill in missing Lat, Long, FIPS
geo_AR_fips.loc[geo_AR_fips["FIPS_z"].isnull()] #6

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
605,605,605,605,5028007010,4/4/20,669,PPP,"C B KING MEMORIAL SCHOOL, INC",1402 S 1ST ST,MC GEHEE,...,Y,933939.40,8/17/21,61.0,Educational Services,ARKANSAS DISTRICT OFFICE,"1402 S 1ST ST, MC GEHEE, AR",30.250536,-97.755145,NaN
3039,3039,3039,3039,2991708608,3/16/21,669,PPS,WRD INC,720 Ramsey St,Batesville,...,N,274269.52,6/23/21,33.0,Manufacturing,ARKANSAS DISTRICT OFFICE,"720 Ramsey St, Batesville, AR",NaN,NaN,NaN
3091,3091,3091,3091,8762207009,4/8/20,669,PPP,"E & M CONTRACTORS, LLC","E & M Contractors, LLC",PO Box 9287,...,N,270255.75,11/27/20,23.0,Construction,ARKANSAS DISTRICT OFFICE,"E & M Contractors, LLC, PO Box 9287, AR",40.021671,-75.946420,NaN
4316,4316,4316,4316,7127367007,4/7/20,669,PPP,FARLEY'S INCORPORATED,19190 AR HIGHWAY 59,SILOAM SPRINGS,...,N,200133.63,1/12/21,33.0,Manufacturing,ARKANSAS DISTRICT OFFICE,"19190 AR HIGHWAY 59, SILOAM SPRINGS, AR",40.110548,-95.145778,NaN
4672,4672,4672,4672,8811267002,4/8/20,669,PPP,"CANE CREEK QUARRY, LLC",2179 CR 321,Poplar Bluff,...,N,174626.56,12/9/20,21.0,Mining,ARKANSAS DISTRICT OFFICE,"2179 CR 321, Poplar Bluff, AR",36.682890,-90.465882,NaN
5177,5177,5177,5177,4507858300,1/23/21,669,PPS,CATADORES JRR INC,528 Central Ave Ste 100,Hot Springs National Park,...,N,NaN,NaN,72.0,Accomodation and Food Services,ARKANSAS DISTRICT OFFICE,"528 Central Ave Ste 100, Hot Springs National ...",NaN,NaN,NaN


In [4]:
geo_AR_fips["BusinessType"].unique()

array(['Corporation', '501(c)3 – Non Profit', 'Professional Association',
       'Subchapter S Corporation', 'Limited  Liability Company(LLC)',
       'Non-Profit Organization', 'Sole Proprietorship',
       'Employee Stock Ownership Plan(ESOP)', 'Partnership',
       'Limited Liability Partnership', 'Cooperative',
       'Non-Profit Childcare Center', 'Trust',
       '501(c)6 – Non Profit Membership', 'Self-Employed Individuals',
       'Independent Contractors'], dtype=object)